### Allow relative imports

In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
import project_path

In [48]:
import glob
import os

from pathlib import Path

import numpy as np
import pandas as pd

from altair_saver import save

from src.plotting import make_v_measure_plot, make_full_plot, graphic_settings

In [14]:
pd.read_csv('../experiments.csv')

,id,Type of experiment
0,0,Local experiments
1,1587641195,KL vs MF algorithms for SBM with 50 cuts
2,1587706041,KL vs MF algorithms for SBM with 50 cuts for ...
3,1587703699,KnnBlobs experiments with different k and 50 ...
4,1586946592,SBM experiments
5,1587639002,SBM with 100 cuts fixed p and q testing diffe...
6,1587621343,KnnBlobs with 100 cuts testing different a
7,1587629564,KL vs MF algorithms on SBM
8,1587632158,SBM with fixed p and q and different numbers ...
9,1587632375,KnnBlobs with fixed k and different numbers o...


In [51]:
experiment_name = '1587723598'

In [52]:
path_in = Path(f'../output/{experiment_name}').resolve()
path_out = Path(f'../plots/{experiment_name}').resolve()
path_out.mkdir(parents=True, exist_ok=True)

In [53]:
full_df = pd.DataFrame()
for subdir, dirs, files in os.walk(path_in):
    for file in files:
        
        current_df = pd.read_csv(f'{subdir}/{file}', index_col=0)
        full_df = full_df.append(current_df)

In [54]:
full_df

,agreement,completeness,dataset_name,dataset_type,homogeneity,max_order,mindset_sizes,nb_questions,nb_useless,noise,order_best,percentile_orders,preprocessing_name,seed,unique_id,v_measure_score
0,30.0,1.00000,mindsets,discrete,1.000000,5.47,"[50, 100]",50.0,8.0,0.1,4.24,100.0,features,42.0,1587723598,1.000000
0,30.0,1.00000,mindsets,discrete,1.000000,3.23,"[80, 100]",50.0,22.0,0.2,3.11,100.0,features,42.0,1587723598,1.000000
0,30.0,1.00000,mindsets,discrete,1.000000,6.00,"[70, 100]",50.0,13.0,0.0,3.78,100.0,features,42.0,1587723598,1.000000
0,30.0,1.00000,mindsets,discrete,1.000000,5.16,"[80, 100]",50.0,18.0,0.0,3.45,100.0,features,42.0,1587723598,1.000000
0,30.0,1.00000,mindsets,discrete,1.000000,3.72,"[70, 100]",50.0,14.0,0.2,3.53,100.0,features,42.0,1587723598,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,30.0,1.00000,mindsets,discrete,1.000000,3.82,"[100, 100, 100]",50.0,13.0,0.1,3.46,100.0,features,42.0,1587723598,1.000000
0,30.0,0.86563,mindsets,discrete,0.992352,3.43,"[100, 100, 100, 100]",50.0,12.0,0.2,3.38,100.0,features,42.0,1587723598,0.924669
0,30.0,1.00000,mindsets,discrete,1.000000,4.00,"[100, 100, 100, 100, 100]",50.0,5.0,0.1,3.72,100.0,features,42.0,1587723598,1.000000
0,30.0,1.00000,mindsets,discrete,1.000000,4.02,"[60, 100]",50.0,30.0,0.0,2.85,100.0,features,42.0,1587723598,1.000000


### Plot Mindsets with variable nb question id = 1587726047

In [55]:
if experiment_name == '1587726047':
    experiments = [x for _, x in full_df.groupby(['mindset_sizes'])]
    for experiment in experiments:
        
        data = experiment
        data['order'] = data['order_best'] / data['max_order']
        data['order'] = data['order'].round(2)
        name = f"mindset_sizes_{data['mindset_sizes'].iloc[0]}"
        title = f"Mindset sizes: {data['mindset_sizes'].iloc[0]} with 50 questions"
        
        chart = make_full_plot(data, title, x_axis='nb_questions', y_axis='noise', facet_on='preprocessing_name')
        save(chart, f'{path_out / name}.svg')

### Plot Mindsets with fixed nb question id = 1587723598

In [57]:
if experiment_name == '1587723598':
    experiments = [x for _, x in full_df.groupby(['mindset_sizes'])]
    for experiment in experiments:
        
        data = experiment
        data['order'] = data['order_best'] / data['max_order']
        data['order'] = data['order'].round(2)
        name = f"mindset_sizes_{data['mindset_sizes'].iloc[0]}"
        title = f"Mindset sizes: {data['mindset_sizes'].iloc[0]} with 50 questions"
        
        chart = make_full_plot(data, title, x_axis='nb_useless', y_axis='noise', facet_on='preprocessing_name')

        save(chart, f'{path_out / name}.svg')

### Plot Mindsets with fixed nb question id = 1587716525

In [32]:
if experiment_name == '1587716525':
    experiments = [x for _, x in full_df.groupby(['mindset_sizes'])]
    for experiment in experiments:
        
        data = experiment
        data['order'] = data['order_best'] / data['max_order']
        data['order'] = data['order'].round(2)
        name = f"mindset_sizes_{data['mindset_sizes'].iloc[0]}"
        title = f"Mindset sizes: {data['mindset_sizes'].iloc[0]} with 50 cuts"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('noise', type='ordinal', sort=alt.EncodingSortField(field='noise', order='descending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('noise', type='ordinal', sort=alt.EncodingSortField(field='noise', order='descending'), axis=alt.Axis(grid=True)),
            alt.Text('order'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='best_order/max_order'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot KnnBlobs with variable k for 50 cuts id = 1587703699

In [19]:
if experiment_name == '1587703699':
    experiments = [x for _, x in full_df.groupby(['blob_centers'])]
    for experiment in experiments:
        
        data = experiment
        name = f"centers_{data['blob_centers'].iloc[0]}"
        title = f"block sizes: {data['blob_centers'].iloc[0]} with 50 cuts"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('k', type='ordinal', sort=alt.EncodingSortField(field='k', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('k', type='ordinal', sort=alt.EncodingSortField(field='k', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot SBM with variable p and q (no 4 blocks) id = 1587641195

In [13]:
if experiment_name == '1587641195':
    full_df = full_df[full_df['block_sizes'] != '[100, 100, 100, 100]']
    experiments = [x for _, x in full_df.groupby(['block_sizes', 'lb_f'])]
    for experiment in experiments:
        
        data = experiment
        name = f"block_sizes_{data['block_sizes'].iloc[0]}_lbf_{data['lb_f'].iloc[0]}"
        title = f"block sizes: {data['block_sizes'].iloc[0]} with lb_f: {data['lb_f'].iloc[0]}"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('p', type='ordinal', sort=alt.EncodingSortField(field='p', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('q', type='ordinal', sort=alt.EncodingSortField(field='q', order='descending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('p', type='ordinal', sort=alt.EncodingSortField(field='p', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('q', type='ordinal', sort=alt.EncodingSortField(field='q', order='descending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot SBM with fixed p and q and different numbers of cuts id = 1587632158

In [60]:
if experiment_name == '1587632158':
    experiments = [x for _, x in full_df.groupby(['block_sizes'])]
    for experiment in experiments:
        
        data = experiment
        name = f"block_sizes_{data['block_sizes'].iloc[0]}"
        title = f"block sizes: {data['block_sizes'].iloc[0]} with p=.3 and q=.1"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='descending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='descending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot KnnBlobs with fixed k and different numbers of cuts id = 1587632375

In [51]:
if experiment_name == '1587632375':
    experiments = [x for _, x in full_df.groupby(['blob_centers'])]
    for experiment in experiments:
        
        data = experiment
        name = f"centers_{data['blob_centers'].iloc[0]}"
        title = f"block sizes: {data['blob_centers'].iloc[0]} with k=15"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot different a for Knn id = 1587621343

In [12]:
if experiment_name == '1587621343':
    experiments = [x for _, x in full_df.groupby(['blob_centers'])]
    for experiment in experiments:
        
        data = experiment
        name = f"centers_{data['blob_centers'].iloc[0]}"
        title = f"block sizes: {data['blob_centers'].iloc[0]} with k=15"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )

        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)

        save(chart, f'{path_out / name}.svg')

### Plot different a for SBM id = 1587616865

In [36]:
if experiment_name == '1587616865':
    
    full_df['order_best'] = full_df[['order_max', 'order_best']].max(axis=1)
    
    experiments = [x for _, x in full_df.groupby(['block_sizes'])]
    for experiment in experiments:

        data = experiment

        name = f"block_sizes_{data['block_sizes'].iloc[0]}"
        title = f"block sizes: {data['block_sizes'].iloc[0]} with p=.3 and q=.1"

        v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='descending'), axis=alt.Axis(grid=True)),
            alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title=title
        )

        text = alt.Chart(data, width=400, height=150).mark_text().encode(
            alt.X('agreement', type='ordinal', sort=alt.EncodingSortField(field='agreement', order='ascending'), axis=alt.Axis(grid=True)),
            alt.Y('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='descending'), axis=alt.Axis(grid=True)),
            alt.Text('order_best'),
        ).facet(
            facet=alt.Facet('preprocessing_name:N', title=None),
            title='order of the best v-measure'
        )
            
        chart = alt.vconcat(v_measure_chart, text)
        chart = graphic_settings(chart)
        
        save(chart, f'{path_out / name}.svg')

In [43]:
experiments = [x for _, x in full_df.groupby(['block_sizes'])]
for experiment in experiments:
    name = f"block_sizes_{data['block_sizes'].iloc[0]}_a_{data['agreement'].iloc[0]}"
    title = f"block sizes: {data['block_sizes'].iloc[0]} with agreement: {data['agreement'].iloc[0]}"

    data = experiment[experiment.nb_cuts == 100]
    base1 = alt.Chart(data, width=400, height=150).mark_rect().encode(
        alt.X('p', type='ordinal', sort=alt.EncodingSortField(field='p', order='ascending'), axis=alt.Axis(grid=True)),
        alt.Y('q', type='ordinal', sort=alt.EncodingSortField(field='q', order='descending'), axis=alt.Axis(grid=True)),
        alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
    ).facet(
        facet=alt.Facet('preprocessing_name:N', title=None),
        title=f'nb cuts: {data.nb_cuts.iloc[0]}'
    )

    data = experiment[experiment.nb_cuts == 200]
    base2= alt.Chart(data, width=400, height=150).mark_rect().encode(
        alt.X('p', type='ordinal', sort=alt.EncodingSortField(field='p', order='ascending'), axis=alt.Axis(grid=True)),
        alt.Y('q', type='ordinal', sort=alt.EncodingSortField(field='q', order='descending'), axis=alt.Axis(grid=True)),
        alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
    ).facet(
        facet=alt.Facet('preprocessing_name:N', title=None),
        title=f'nb cuts: {data.nb_cuts.iloc[0]}'
    )

    data = experiment[experiment.nb_cuts == 300]
    base3 = alt.Chart(data, width=400, height=150).mark_rect().encode(
        alt.X('p', type='ordinal', sort=alt.EncodingSortField(field='p', order='ascending'), axis=alt.Axis(grid=True)),
        alt.Y('q', type='ordinal', sort=alt.EncodingSortField(field='q', order='descending'), axis=alt.Axis(grid=True)),
        alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
    ).facet(
        facet=alt.Facet('preprocessing_name:N', title=None),
        title=f'nb cuts: {data.nb_cuts.iloc[0]}'
    )

    chart = alt.vconcat()
    chart = base1 & base2 & base3
    chart = chart.properties(
        title=title
    )

    chart = chart.configure_title(
            fontSize=10,
            font='Courier',
            anchor='middle',
            color='gray'
        ).configure_axis(
            gridOpacity = 0.0,

            labelFont='Courier',
            labelColor='black',

            titleFont='Courier',
            titleColor='gray',
            grid=False
        ).configure_axisX(
            labelAngle=0,
        ).configure_axisY(
            titleAngle=0,
            titlePadding=10,
        ).configure_legend(
            labelFont='Courier',
            labelColor='black',

            titleFont='Courier',
            titleColor='gray',
            titleAnchor='middle'
        ).configure_view(strokeOpacity=0)
    save(chart, f'{path_out / name}.svg')

### Plot comparison between preprocessing methods id = 1586947660

In [15]:
experiments = [x for _, x in full_df.groupby(['block_sizes', 'agreement'])]
for experiment in experiments:
    data = experiment

    name = f"block_sizes_{data['block_sizes'].iloc[0]}_a_{data['agreement'].iloc[0]}"
    title = f"block sizes: {data['block_sizes'].iloc[0]} with agreement: {data['agreement'].iloc[0]}"

    v_measure_chart = alt.Chart(data, width=400, height=150).mark_rect().encode(
        alt.Y('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='descending'), axis=alt.Axis(grid=True)),
        alt.X('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
        alt.Color('v_measure_score', type='quantitative', title='v-measure score',scale=alt.Scale(domain=[0, 1])),
    ).facet(
        facet=alt.Facet('preprocessing_name:N', title=None),
        title=title
    )

    text = alt.Chart(data, width=400, height=150).mark_text().encode(
        alt.Y('nb_cuts', type='ordinal', sort=alt.EncodingSortField(field='nb_cuts', order='descending'), axis=alt.Axis(grid=True)),
        alt.X('lb_f', type='ordinal', sort=alt.EncodingSortField(field='lb_f', order='ascending'), axis=alt.Axis(grid=True)),
        alt.Text('order_max'),
    ).facet(
        facet=alt.Facet('preprocessing_name:N', title=None),
        title='max order'
    )
    chart = alt.vconcat(v_measure_chart, text)
    chart = chart.configure_title(
            fontSize=10,
            font='Courier',
            anchor='middle',
            color='gray'
        ).configure_axis(
            gridOpacity = 0.0,

            labelFont='Courier',
            labelColor='black',

            titleFont='Courier',
            titleColor='gray',
            grid=False
        ).configure_axisX(
            labelAngle=0,
        ).configure_axisY(
            titleAngle=0,
            titlePadding=30,
        ).configure_legend(
            labelFont='Courier',
            labelColor='black',

            titleFont='Courier',
            titleColor='gray',
            titleAnchor='middle'
        ).configure_view(strokeOpacity=0)
    save(chart, f'{path_out / name}.svg')